#### Produce constructor standings

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")
v_file_date

Out[68]: '2021-03-28'

In [0]:
path = f"{presentation_folder_path}/race_results"
race_results_df = column_to_list(path, 'race_year',v_file_date)
display(race_results_df)

race_id,race_year,race_name,race_date,circuit_location,driver_name,driver_number,driver_nationality,team,grid,fastest_lap,race_time,points,position,file_date,created_date
1053,2021,Emilia Romagna Grand Prix,2021-04-18T13:00:00.000+0000,Imola,Mick Schumacher,null,German,Haas F1 Team,18,58,\N,0.0,16,2021-04-18,2024-03-17T23:03:26.540+0000
1053,2021,Emilia Romagna Grand Prix,2021-04-18T13:00:00.000+0000,Imola,Nikita Mazepin,null,Russian,Haas F1 Team,19,55,\N,0.0,17,2021-04-18,2024-03-17T23:03:26.540+0000
1053,2021,Emilia Romagna Grand Prix,2021-04-18T13:00:00.000+0000,Imola,Yuki Tsunoda,null,Japanese,AlphaTauri,20,49,+73.184,0.0,12,2021-04-18,2024-03-17T23:03:26.540+0000
1053,2021,Emilia Romagna Grand Prix,2021-04-18T13:00:00.000+0000,Imola,Nicholas Latifi,6,Canadian,Williams,14,null,\N,0.0,null,2021-04-18,2024-03-17T23:03:26.540+0000
1053,2021,Emilia Romagna Grand Prix,2021-04-18T13:00:00.000+0000,Imola,George Russell,63,British,Williams,12,28,\N,0.0,null,2021-04-18,2024-03-17T23:03:26.540+0000
1053,2021,Emilia Romagna Grand Prix,2021-04-18T13:00:00.000+0000,Imola,Lando Norris,4,British,McLaren,7,63,+23.702,15.0,3,2021-04-18,2024-03-17T23:03:26.540+0000
1053,2021,Emilia Romagna Grand Prix,2021-04-18T13:00:00.000+0000,Imola,Charles Leclerc,16,Monegasque,Ferrari,4,60,+25.579,12.0,4,2021-04-18,2024-03-17T23:03:26.540+0000
1053,2021,Emilia Romagna Grand Prix,2021-04-18T13:00:00.000+0000,Imola,Pierre Gasly,10,French,AlphaTauri,5,52,+52.818,6.0,7,2021-04-18,2024-03-17T23:03:26.540+0000
1053,2021,Emilia Romagna Grand Prix,2021-04-18T13:00:00.000+0000,Imola,Antonio Giovinazzi,99,Italian,Alfa Romeo,17,57,\N,0.0,14,2021-04-18,2024-03-17T23:03:26.540+0000
1053,2021,Emilia Romagna Grand Prix,2021-04-18T13:00:00.000+0000,Imola,Lance Stroll,18,Canadian,Aston Martin,10,59,+56.909,4.0,8,2021-04-18,2024-03-17T23:03:26.540+0000


In [0]:
from pyspark.sql.functions import sum, count, when, col

In [0]:
constructor_standings_df = race_results_df \
.groupBy("race_year", "team") \
.agg(sum("points").alias("total_points"), count(when(col("position") == 1, True)).alias("wins"))
display(constructor_standings_df.filter("race_year = 2020"))

race_year,team,total_points,wins


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank

In [0]:
constructor_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"), desc("wins"))
final_df = constructor_standings_df.withColumn("rank", rank().over(constructor_rank_spec))

In [0]:
# final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/constructor_standings")
# final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.constructor_standings")
# overwrite_partition(final_df, 'f1_presentation', 'constructor_standings', 'race_year')

merge_condition = "tgt.team = src.team AND tgt.race_year = src.race_year"
merge_delta_data(final_df, "f1_presentation", 'constructor_standings', presentation_folder_path, merge_condition, 'race_year')

In [0]:
%sql
SELECT *
FROM f1_presentation.driver_standings WHERE race_year = 2021

race_year,driver_name,driver_nationality,total_points,wins,rank
2021,Lewis Hamilton,British,44.0,1,1
2021,Max Verstappen,Dutch,43.0,1,2
2021,Lando Norris,British,27.0,0,3
2021,Charles Leclerc,Monegasque,20.0,0,4
2021,Valtteri Bottas,Finnish,16.0,0,5
2021,Daniel Ricciardo,Australian,14.0,0,6
2021,Carlos Sainz,Spanish,14.0,0,6
2021,Sergio Pérez,Mexican,10.0,0,8
2021,Pierre Gasly,French,6.0,0,9
2021,Lance Stroll,Canadian,5.0,0,10
